In [13]:
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from datetime import datetime
import threading
lock = threading.RLock()

In [14]:
rawVideos = pd.read_pickle('../datasets/rawVideosContent.pkl')
print(len(rawVideos))
rawVideos.head(3)

36657


,videoId,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate
U6M5iX9qWh8,U6M5iX9qWh8,2,0,437,"[TEDxTalks, English, Egypt, Social Science, Bu...",332,NaN,Deena Bahgat is only 21 but runs her own busin...,1.57697e+09
qNvKWhzdpW4,qNvKWhzdpW4,0,0,1001,"[TEDxTalks, English, Life, Anthropology, Compl...",222,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Dr. Kyle William Bishop is a Cedar City native...,1.57697e+09
huOxk-UhOz0,huOxk-UhOz0,468,4,1161,"[TEDxTalks, English, United States, Life, Fail...",26243,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","With an honest and vulnerable approach, Joey H...",1.57697e+09


In [15]:
videos = pd.read_pickle('../datasets/videosData.pkl')
print(len(rawVideos))
videos.head(3)

36657


,title,speaker,event,videoId,titleRaw,publishedTimeRaw,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,text,publishDate
-eLgCDFKD7s,The Art of living with Leopards,Nikit Surve,TEDxVileParle,-eLgCDFKD7s,The Art of living with Leopards | Nikit Surve ...,1 week ago,10,0,1006,"[TEDxTalks, English, Global Issues, Animals]",544,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Nikit talks about wildlife and his work within...,1.57697e+09,I don't have never was in Mumbai so muchall of...,2019-05-17 00:00:00
DF4ipJE_t_A,Paisajes invisibles,Francisco García & Manuel Bugallo,TEDxGalicia,DF4ipJE_t_A,Paisajes invisibles | Francisco García & Manue...,1 week ago,21,0,723,"[TEDxTalks, Spanish, Global Issues, Environment]",761,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Fran y Manuel nos ofrecen una mirada diferente...,1.57699e+09,I always loved the Mediterranean maybe because...,2019-05-17 00:00:00
587FCkioktM,Ce que j’ai appris de 15 000km à vélo,Pierre Bouillot,TEDxINSA,587FCkioktM,Ce que j’ai appris de 15 000km à vélo | Pierre...,1 week ago,6,0,890,"[TEDxTalks, French, Life, Adventure, Explorati...",283,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","Diplômé de l'INSA Lyon en 2016, Pierre Bouillo...",1.57699e+09,we're fine great started you go open your eye...,2019-05-17 00:00:00


In [20]:
# rawVideos.loc[videos.index.values, :]

rawVideos = rawVideos[rawVideos.index.isin(videos.index.values)]

In [24]:
rawVideos.loc[:, 'publishDate'] = videos.loc[rawVideos.index].publishDate

In [27]:
rawVideos

,videoId,likes,dislikes,lengthSeconds,keywords,views,talkText,videoDescription,currentDate,publishDate
qNvKWhzdpW4,qNvKWhzdpW4,0,0,1001,"[TEDxTalks, English, Life, Anthropology, Compl...",222,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Dr. Kyle William Bishop is a Cedar City native...,1.57697e+09,2019-05-15
huOxk-UhOz0,huOxk-UhOz0,468,4,1161,"[TEDxTalks, English, United States, Life, Fail...",26243,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","With an honest and vulnerable approach, Joey H...",1.57697e+09,2016-05-17
J0_6b-AJdWU,J0_6b-AJdWU,21,1,764,"[TEDxTalks, English, Health, Communication, Cu...",1085,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",How limited and limiting do our lives become w...,1.57697e+09,2018-05-22
QjVNcAAf7pA,QjVNcAAf7pA,150,72,927,"[TEDxTalks, English, Social Science, Communica...",12004,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",America has a crisis of conspiracy theory beli...,1.57697e+09,2019-05-16
kP8wJvBXHgs,kP8wJvBXHgs,290,20,685,"[TEDxTalks, English, Life, Change, Youth]",40182,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Guro Heggholmen (11) has a special interest in...,1.57697e+09,2017-10-17
nhL1S2_zejc,nhL1S2_zejc,20,0,804,"[South Africa, systems biology, healthcare, mh...",1612,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Riaan offers some of the practical changes nee...,1.57697e+09,2013-09-23
B8QiEK-fgHM,B8QiEK-fgHM,4,1,1105,"[TEDxTalks, English, United States, Humanities...",417,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Archaeological projects like those directed by...,1.57697e+09,2016-09-14
Q_A5MaTBejE,Q_A5MaTBejE,20,1,1053,"[Exploration, ted, Planets, Astronomy, tedx ta...",1446,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...","In the spirit of ideas worth spreading, TEDx i...",1.57697e+09,2013-10-23
UWiBqcZK8oo,UWiBqcZK8oo,17,1,629,"[TEDxTalks, English, United States, Life, Comm...",1266,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",Sean’s TEDxNorthwesternU talk centers around t...,1.57697e+09,2016-04-15
sSeEQUYAAOk,sSeEQUYAAOk,35,0,478,"[TEDxTalks, English, Global Issues, Energy, En...",680,"<?xml version=""1.0"" encoding=""utf-8"" ?><transc...",I have worked in development/global education ...,1.57697e+09,2019-04-17


In [28]:
rawVideos.to_pickle('../datasets/rawVideosContent.pkl')